In [43]:
from pettingzoo.classic import connect_four_v3
import numpy as np
import matplotlib.pyplot as plt
import gymnasium as gym
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
from cart_pole import DQN, ReplayMemory, Transition, train

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import os
os.environ["SDL_VIDEODRIVER"] = "dummy"
from IPython.display import clear_output

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [44]:
env = connect_four_v3.env(render_mode="rgb_array")

In [45]:
def state_observation_to_DQN_input(state_observation, agent_selection, current_player, my_value=1, ennemy_value=2):
    if current_player == agent_selection : 
        my_pieces = 0
        adverse_pieces = 1
    else : 
        my_pieces = 1
        adverse_pieces = 0
    result = np.zeros(state_observation.shape[:2])
    result[np.where(state_observation[:, :, my_pieces] == 1)] = my_value
    result[np.where(state_observation[:, :, adverse_pieces] == 1)] = ennemy_value
    return result.flatten()

env.reset()
env.step(0) # 0 
env.step(6) # 1 
env.step(0) # 0
env.step(6) # 1
env.step(2) # 0
env.step(3)
state, reward, terminated, truncated, info = env.last()
print(env.agent_selection)
state_observation_to_DQN_input(state["observation"], env.agent_selection, 'player_0')

player_0


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       2., 1., 0., 1., 2., 0., 0., 2.])

In [46]:
env.reset()

# The main difference with the standard gym api is the way the environment is queried. The `step` method return `None`. To get the data on the environment, use the `last` method
state, reward, terminated, truncated, info = env.last()

# state is a dictionary with two keys: observation and action_mask
print(
    state["observation"].shape
)  # Observation is a numpy array with three coordinates, indicating the positions of the pieces of of player 0 and 1 on the the board
print(state["observation"][:, :, 0])  # Where the pieces of player 0 are
print(state["observation"][:, :, 1])  # Where the pieces of player 1 are

print(state["action_mask"])  # an array showing whether the actions are legal or nots


(6, 7, 2)
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
[1 1 1 1 1 1 1]


In [47]:
# set up matplotlib
is_ipython = "inline" in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

BATCH_SIZE=128
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 1000
TAU = 0.005
LR = 1e-4

# Get number of actions from gym action space
n_actions = env.action_space(env.possible_agents[0]).n
# Get the number of state observations
env.reset()
state, reward, terminated, truncated, info = env.last()
n_observations = state["observation"].shape[0] * state["observation"].shape[1]

policy_net = DQN(n_observations, n_actions).to(device)
target_net = DQN(n_observations, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())

optimizer = optim.AdamW(policy_net.parameters(), lr=LR, amsgrad=True)
memory = ReplayMemory(10000)

train(
    env,
    BATCH_SIZE,
    GAMMA,
    EPS_START,
    EPS_END,
    EPS_DECAY,
    TAU,
    optimizer,
    memory,
    policy_net,
    target_net,
)

TypeError: cannot unpack non-iterable NoneType object